# Logistic Regression in the Diagnosis of Parkinson's Disease
In this notebook we will implement multiple regression to diagnose parkinson disease, the dataset is obtained from Oxford Parkinson's Disease Detection Dataset (https://archive.ics.uci.edu/dataset/174/parkinsons)

## Tools
In this project these libraries are used: 
- NumPy, a popular library for scientific computing
- Matplotlib, a popular library for plotting data

In [1]:
from utils import *
import copy, math
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Problem Statement

Background:
- Parkinson's disease is a progressive neurodegenerative disorder that affects movement control. Early and accurate diagnosis is crucial for effective management and intervention. Machine learning techniques, such as logistic regression, offer a promising approach to diagnosing Parkinson's disease based on relevant clinical features.

Objective:
- The goal of this project is to implement logistic regression to develop a diagnostic model for Parkinson's disease. Leveraging a dataset containing clinical measurements and relevant features, the model will learn to distinguish between patients with Parkinson's disease and those without.

Dataset:
- The dataset consists of clinical measurements collected from individuals, including features related to voice and motor control. Each example in the dataset is labeled with the binary outcome: whether the individual has been diagnosed with Parkinson's disease (1) or not (0).

Begin by looking at the structure of the csv that contains the data:

In [2]:
PARKINSONS_CSV = './data/PARKINSONS.csv'

with open(PARKINSONS_CSV, 'r') as csvfile:
    print(f"Header looks like this:\n\n{csvfile.readline()}")    
    print(f"First data point looks like this:\n\n{csvfile.readline()}")
    print(f"Second data point looks like this:\n\n{csvfile.readline()}")

Header looks like this:

name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,MDVP:APQ,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE

First data point looks like this:

phon_R01_S01_1,119.99200,157.30200,74.99700,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.42600,0.02182,0.03130,0.02971,0.06545,0.02211,21.03300,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654

Second data point looks like this:

phon_R01_S01_2,122.40000,148.65000,113.81900,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.62600,0.03134,0.04518,0.04368,0.09403,0.01929,19.08500,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674



## Dataset
### Information
- This dataset is composed of a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD). Each column in the table is a particular voice measure, and each row corresponds one of 195 voice recording from these individuals ("name" column). The main aim of the data is to discriminate healthy people from those with PD, according to "status" column which is set to 0 for healthy and 1 for PD. 

- The data is in ASCII CSV format. The rows of the CSV file contain an instance corresponding to one voice recording. There are around six recordings per patient, the name of the patient is identified in the first column.For further information or to pass on comments, please contact Max Little (littlem '@' robots.ox.ac.uk).

- Reference:

  Max A. Little, Patrick E. McSharry, Eric J. Hunter, Lorraine O. Ramig (2008), 'Suitability of dysphonia measurements for telemonitoring of Parkinson's disease', IEEE Transactions on Biomedical Engineering

### Load data
- The `load_data()` function shown below loads the data into variables `X_train` and `y_train`
  - `X_train` is a range of biomedical voice measurements from 31 people, 23 with Parkinson's disease (PD)
  - `y_train` is whether or not the person has Parkinson's disease
      - `y_train = 1` if the disease is diagnosed
      - `y_train = 0` if the disease is not diagnosed
  - Both `X_train` and `y_train` are numpy arrays.

In [3]:
X_train, y_train = load_data()

#### View the variables and Dimension
The code below prints the variable `x_train` and the type of the variable.

In [4]:
# print x_train
print(f"X Shape: {X_train.shape}, X Type:{type(X_train)})")
print("Type of x_train:",type(X_train))
print("First three elements of x_train are:\n", X_train[:3]) 

X Shape: (195, 22), X Type:<class 'numpy.ndarray'>)
Type of x_train: <class 'numpy.ndarray'>
First three elements of x_train are:
 [[ 1.199920e+02  1.573020e+02  7.499700e+01  7.840000e-03  7.000000e-05
   3.700000e-03  5.540000e-03  1.109000e-02  4.374000e-02  4.260000e-01
   2.182000e-02  3.130000e-02  2.971000e-02  6.545000e-02  2.211000e-02
   2.103300e+01  4.147830e-01  8.152850e-01 -4.813031e+00  2.664820e-01
   2.301442e+00  2.846540e-01]
 [ 1.224000e+02  1.486500e+02  1.138190e+02  9.680000e-03  8.000000e-05
   4.650000e-03  6.960000e-03  1.394000e-02  6.134000e-02  6.260000e-01
   3.134000e-02  4.518000e-02  4.368000e-02  9.403000e-02  1.929000e-02
   1.908500e+01  4.583590e-01  8.195210e-01 -4.075192e+00  3.355900e-01
   2.486855e+00  3.686740e-01]
 [ 1.166820e+02  1.311110e+02  1.115550e+02  1.050000e-02  9.000000e-05
   5.440000e-03  7.810000e-03  1.633000e-02  5.233000e-02  4.820000e-01
   2.757000e-02  3.858000e-02  3.590000e-02  8.270000e-02  1.309000e-02
   2.065100e+01

In [5]:
# print y_train
print(f"y Shape: {y_train.shape}, y Type:{type(y_train)})")
print("Type of y_train:",type(y_train))
print("First five elements of y_train are:\n", y_train[:5])  

y Shape: (195,), y Type:<class 'numpy.ndarray'>)
Type of y_train: <class 'numpy.ndarray'>
First five elements of y_train are:
 [1 1 1 1 1]


# Visualizing Data

Before starting to implement the learning algorithm, it is always good to visualize the data if possible.
- The code below displays the data on a 2D plot (as shown below), where the axes are y label features.
- A helper function in the ``utils.py`` file to generate this plot. 

### Sigmoid function

Logistic regression model is represented as

$$f_{\mathbf{w},b}(x) = g(\mathbf{w}\cdot \mathbf{x} + b)$$
where function $g$ is the sigmoid function. The sigmoid function is defined as:

$$g(z) = \frac{1}{1+e^{-z}}$$

Note that 
- `z` is not always a single number, but can also be an array of numbers. 
- If the input is an array of numbers, it is good to apply the sigmoid function to each value in the input array.

In [6]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar, numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
         
    """
    g = 1/(1 + np.exp(-z))
    
    return g

- For large positive values of x, the sigmoid should be close to 1, while for large negative values, the sigmoid should be close to 0. 
- Evaluating `sigmoid(0)` should give you exactly 0.5. 

In [7]:
#You can edit this value
value = 0

print (f"sigmoid({value}) = {sigmoid(value)}")

sigmoid(0) = 0.5


<a name="2.4"></a>
### Cost function for logistic regression

The `compute_cost` function using the equations below.

Logistic regression cost function is of the form 

$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

where
* m is the number of training examples in the dataset


* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ is the cost for a single data point, which is - 

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$
    
    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$, which is the actual label

*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x^{(i)}} + b)$ where function $g$ is the sigmoid function.

In [8]:
def compute_cost(X, y, w, b, *argv):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns:
      total_cost : (scalar) cost 
    """

    m, n = X.shape
    
    cost = 0.
    epsilon = 1e-15  # Small constant to avoid log(0) and division by zero
    
    for i in range(m):
        z_i = np.dot(X[i], w) + b
        f_wb_i = sigmoid(z_i)
        # Avoiding extreme values to prevent NaN issues
        f_wb_i = np.clip(f_wb_i, epsilon, 1 - epsilon)
        cost += - y[i] * np.log(f_wb_i) - (1 - y[i]) * np.log(1 - f_wb_i)
    
    total_cost = cost / m

    return total_cost

Cell below is to check the implementation of the `compute_cost` function with two different initializations of the parameters $w$ and $b$

In [9]:
m, n = X_train.shape

# Compute and display cost with w and b initialized to zeros
initial_w = np.zeros(n)
initial_b = 0.
cost = compute_cost(X_train, y_train, initial_w, initial_b)
print('Cost at initial w and b (zeros): {:.3f}'.format(cost))

Cost at initial w and b (zeros): 0.693


### Gradient for logistic regression

Gradient descent algorithm is:

$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & b := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline       \; & w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1}\newline & \rbrace\end{align*}$$

where, parameters $b$, $w_j$ are all updated simultaniously

This `compute_gradient` function is to compute $\frac{\partial J(\mathbf{w},b)}{\partial w}$, $\frac{\partial J(\mathbf{w},b)}{\partial b}$ from equations (2) and (3) below.

$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$
* m is the number of training examples in the dataset

    
*  $f_{\mathbf{w},b}(x^{(i)})$ is the model's prediction, while $y^{(i)}$ is the actual label


- **Note**: While this gradient looks identical to the linear regression gradient, the formula is actually different because linear and logistic regression have different definitions of $f_{\mathbf{w},b}(x)$.

In [10]:
def compute_gradient(X, y, w, b, *argv): 
    """
    Computes the gradient for logistic regression 
 
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value 
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w. 
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b. 
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.

    for i in range(m):
        z_wb = np.dot(X[i], w) + b
        f_wb = sigmoid(z_wb)
        
        dj_db_i = f_wb - y[i]
        dj_db += dj_db_i
        
        for j in range(n): 
            dj_dw[j] += X[i, j] * dj_db_i
            
    dj_db /= m
    dj_dw /= m
        
    return dj_db, dj_dw

Cells below is to check the implementation of the `compute_gradient` function with two different initializations of the parameters $w$ and $b$

In [11]:
# Compute and display gradient with w and b initialized to zeros
initial_w = np.zeros(n)
initial_b = 0.

dj_db, dj_dw = compute_gradient(X_train, y_train, initial_w, initial_b)
print(f'dj_db at initial w and b (zeros):{dj_db}' )
print(f'dj_dw at initial w and b (zeros):{dj_dw.tolist()}' )

dj_db at initial w and b (zeros):-0.25384615384615383
dj_dw at initial w and b (zeros):[-32.32963846153845, -43.50341282051283, -22.41898205128205, -0.002158589743589745, -1.6225641025641027e-05, -0.001179358974358975, -0.0012170769230769227, -0.003538179487179484, -0.010518512820512813, -0.10101282051282048, -0.005492743589743587, -0.006352410256410258, -0.008765717948717945, -0.016478102564102568, -0.009597025641025648, -4.868217948717946, -0.1403319230769231, -0.18779646153846152, 1.1783795717948717, -0.07379868205128204, -0.6605768641025636, -0.07299468717948715]


### Learning parameters using gradient descent 

Finding the optimal parameters of a logistic regression model by using gradient descent. 

- A good way to verify that gradient descent is working correctly is to look
at the value of $J(\mathbf{w},b)$ and check that it is decreasing with each step. 

- The value of $J(\mathbf{w},b)$ should never increase, and should converge to a steady value by the end of the algorithm.

In [12]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_): 
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      X :    (ndarray Shape (m, n) data, m examples by n features
      y :    (ndarray Shape (m,))  target value 
      w_in : (ndarray Shape (n,))  Initial values of parameters of the model
      b_in : (scalar)              Initial value of parameter of the model
      cost_function :              function to compute cost
      gradient_function :          function to compute gradient
      alpha : (float)              Learning rate
      num_iters : (int)            number of iterations to run gradient descent
      lambda_ : (scalar, float)    regularization constant
      
    Returns:
      w : (ndarray Shape (n,)) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    
    # number of training examples
    m = len(X)
    
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    w_history = []
    
    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)   

        # Update Parameters using w, b, alpha and gradient
        w_in = w_in - alpha * dj_dw               
        b_in = b_in - alpha * dj_db              
       
        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            cost =  cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0 or i == (num_iters-1):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
        
    return w_in, b_in, J_history, w_history #return w and J,w history for graphing

Here to see the gradient descent converging

If your gradient descent is not converging, there are a few potential issues to investigate:

Learning Rate (Alpha):
- Experiment with different learning rates. Try reducing the learning rate and observe the effect on convergence.

Feature Scaling:
- Normalize or standardize your features to have a similar scale. This can often help gradient descent converge more efficiently.

Check for Bugs in Cost Function or Gradient Calculation:
- Double-check your cost function and gradient calculations. Print or log intermediate values during training to identify where the issue might be occurring.

Check for Numerical Stability Issues:
- Check for operations like taking the logarithm, exponentiation, or division that may lead to extreme values. Use functions like np.clip to prevent values from going out of a reasonable range.

In [13]:
np.random.seed(1)
initial_w = np.random.randn(22) * 0.01
initial_b = 0

# Some gradient descent settings (can be tweaked/ changed)
iterations = 10000 
alpha = 0.0001

w,b, J_history,_ = gradient_descent(X_train ,y_train, initial_w, initial_b, 
                                   compute_cost, compute_gradient, alpha, iterations, 0)

Iteration    0: Cost     0.65   


Iteration 1000: Cost     0.58   
Iteration 2000: Cost     0.58   
Iteration 3000: Cost     0.57   
Iteration 4000: Cost     0.57   
Iteration 5000: Cost     0.57   
Iteration 6000: Cost     0.56   
Iteration 7000: Cost     0.56   
Iteration 8000: Cost     0.56   
Iteration 9000: Cost     0.55   
Iteration 9999: Cost     0.55   


See the gradient descent converges

In [14]:
def predict(X, w, b): 
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w
    
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model      
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape   
    p = np.zeros(m)
   
    ### START CODE HERE ### 
    # Loop over each example
    for i in range(m):   
        z_wb = np.dot(X[i], w) + b  # Linear combination of features and weights
        f_wb = sigmoid(z_wb)  # Sigmoid function to get probabilities
        
        # Apply the threshold (0.5) to determine the class (0 or 1)
        p[i] = 1 if f_wb > 0.5 else 0
        
    ### END CODE HERE ### 
    return p

In [15]:
#Compute accuracy on our training set
p = predict(X_train, w,b)
print('Train Accuracy: %f'%(np.mean(p == y_train) * 100))

Train Accuracy: 76.410256


## Overfitting